<a href="https://colab.research.google.com/github/AbhishekkTrivedi/SVM_Models_-_Stacking_Ensemble-/blob/main/Stacking_freeplane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from numpy import mean
from numpy import std
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
Abhi = pd.read_csv("/content/gdrive/MyDrive/freeplane 1.7.9.csv")

In [27]:
Abhi['ALTER'] = Abhi['ALTER'].map({'YES': 1, 'NO': 0})

In [28]:
Abhi.shape

(1278, 12)

In [29]:
Abhi.head()

,CountClassCoupled,CountClassDerived,CountDeclInstanceVariable,CountDeclInstanceMethod,CountDeclMethodAll,CountDeclMethodPrivate,CountDeclMethodPublic,CountLineCode,MaxInheritanceTree,PercentLackOfCohesion,SumCyclomatic,ALTER
0,6,0,4,9,12,5,5,155,1,70,32,0
1,9,0,4,5,5,0,4,118,1,50,22,0
2,3,0,1,2,2,0,1,20,2,0,4,0
3,2,0,0,1,1,0,0,7,2,0,1,0
4,0,0,0,14,24,4,9,128,2,0,27,0


In [30]:
X = Abhi.drop('ALTER', axis=1)
y = Abhi['ALTER']

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [32]:
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('sig', SVC(probability=True,kernel='sigmoid')))
	level0.append(('lin', SVC(probability=True,kernel='linear')))
	level0.append(('poly', SVC(probability=True,kernel='poly')))
	level0.append(('rbf', SVC(probability=True,kernel='rbf')))
	# define meta learner model
	level1 = SVC(probability=True,kernel='rbf')
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1)
	return model

In [33]:
def get_models():
	models = dict()
	#models['lin'] = SVC(kernel='linear')
	#models['sig'] = SVC(kernel='sigmoid')
	#models['poly'] = SVC(kernel='poly')
	#models['rbf'] = SVC(kernel='rbf')

	models['stacking'] = get_stacking()
	return models

In [34]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [35]:
models = get_models()

In [36]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))


>stacking 0.000 (0.000)


In [37]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [38]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[230   0]
 [ 26   0]]


In [39]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.6160535117056856


In [40]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.0


In [41]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5


In [42]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.0


In [43]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       230
           1       0.00      0.00      0.00        26

    accuracy                           0.90       256
   macro avg       0.45      0.50      0.47       256
weighted avg       0.81      0.90      0.85       256



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
